Run Iterations all at Once

In [7]:
import importlib

import metaspread
import metaspread.simrunner as simrunner

importlib.reload(metaspread)

import pandas as pd
import random
import time
import os
import ast
import json
import uuid

def run_n_times(number_of_simulations, maximum_steps):
    # Set random seed for reproducibility
    random.seed(int(time.time()))

    # Get the directory where metaspread is installed
    package_dir = os.path.dirname(metaspread.__file__)

    # Path to simulation_configs.csv inside metaspread
    config_path = os.path.join(package_dir, "simulations_configs.csv")

    # Load default parameters from the CSV file
    df_configs = pd.read_csv(config_path, header=0, converters={"Values": ast.literal_eval})
    default_params = dict(zip(df_configs["Names"], df_configs["Values"]))

    # List of parameters to vary
    user_selected_params = ["dM", "dE", "phiM","phiE","mesenchymal_proportion","epithelial_proportion"]  # Replace with actual parameter names
    valid_params = [param for param in user_selected_params if param in default_params]

    # Number of simulations
    num_simulations = number_of_simulations

    # Store all parameter variations
    simulation_logs = []

    # Loop over multiple iterations
    for i in range(num_simulations):
        print(f"Running simulation {i+1}/{num_simulations}...")

        simulation_id = str(uuid.uuid4())[:8]

        # Modify parameters randomly
        varied_params = default_params.copy()
        for param in valid_params:
            if param != "mesenchymal_proportion" and param != "epithelial_proportion":
                default_value = float(default_params[param])  # Ensure it's numeric
                new_value = default_value * random.uniform(1/3, 3)
                varied_params[param] = new_value
            else:
                varied_params[param] = random.uniform(0,1)

        # Ensure grid size is always set to 50
        varied_params["gridsize"] = 50  

        # set mesenchymal_porpotion and ensure the proportions add up to 1
        mes_proportion = random.uniform(0,1)
        varied_params["mesenchymal_proportion"] = mes_proportion
        varied_params["epithelial_proportion"] = 1 - mes_proportion

        # Update the CSV file with modified parameters
        df_configs.loc[df_configs["Names"].isin(valid_params + ["gridsize"]), "Values"] = df_configs["Names"].map(varied_params)
        df_configs.to_csv(config_path, index=False)

        # Run simulation
        simulation = simrunner.run_simulation(simulation_id, max_steps=maximum_steps, data_collection_period=10)

        # Store parameter variations
        simulation_logs.append({
            "simulation_id": simulation_id,
            "varied_parameters": {param: varied_params[param] for param in valid_params},
            "gridsize": 50
        })

        print(f"Completed simulation {i+1}/{num_simulations}.")

    # Save all parameters used in simulations
    with open("metaspread_simulation_logs.json", "w") as f:
        json.dump(simulation_logs, f, indent=4)

    print(f"All {num_simulations} simulations completed successfully.")


## Run
run_n_times({number of simulations}, {maximum steps})

In [8]:
run_n_times(5,30)

Running simulation 1/5...

	 Folder for this simulation: Simulations/Sim-00f64925-Date-2025-03-01_14-11-38
	 Saving agents data at: Simulations/Sim-00f64925-Date-2025-03-01_14-11-38
	 Saving Mmp2 data at: Simulations/Sim-00f64925-Date-2025-03-01_14-11-38/Mmp2
	 Saving Ecm data at: Simulations/Sim-00f64925-Date-2025-03-01_14-11-38/Ecm
	 Saving Vasculature data at: Simulations/Sim-00f64925-Date-2025-03-01_14-11-38/Vasculature
	 Saving all the simulations parameters at: Simulations/Sim-00f64925-Date-2025-03-01_14-11-38/configs.csv
Starting simulation from zero!
Finished the simulation at time step 30!
Completed simulation 1/5.
Running simulation 2/5...

	 Folder for this simulation: Simulations/Sim-43184b7c-Date-2025-03-01_14-11-59
	 Saving agents data at: Simulations/Sim-43184b7c-Date-2025-03-01_14-11-59
	 Saving Mmp2 data at: Simulations/Sim-43184b7c-Date-2025-03-01_14-11-59/Mmp2
	 Saving Ecm data at: Simulations/Sim-43184b7c-Date-2025-03-01_14-11-59/Ecm
	 Saving Vasculature data at: Si